In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

# Suppress specific warnings for cleaner UI
warnings.filterwarnings("ignore")

# --- Library Import Handling ---
HAS_LIB = False
LIB_ERROR_MSG = ""

try:
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM
    HAS_LIB = True
except ImportError as e:
    LIB_ERROR_MSG = str(e)

# --- Interactive UI Class ---
class GPT2App:
    def __init__(self):
        self.tokenizer = None
        self.model = None
        self.is_loaded = False
        
        # --- Widgets Definition ---
        self.output = widgets.Output()
        
        self.loading_bar = widgets.FloatProgress(
            value=0.0,
            min=0.0,
            max=1.0,
            description='Loading:',
            bar_style='info',
            layout=widgets.Layout(width='100%', visibility='hidden')
        )

        self.input_text = widgets.Textarea(
            value='Once upon a time in a distant galaxy',
            placeholder='Type your prompt here...',
            description='Prompt:',
            layout=widgets.Layout(width='98%', height='100px')
        )
        
        self.length_slider = widgets.IntSlider(
            value=50, 
            min=10, 
            max=200, 
            step=5, 
            description='Max Length:', 
            continuous_update=False,
            style={'description_width': 'initial'}
        )
        
        self.btn_generate = widgets.Button(
            description="Generate Text",
            button_style='primary',
            icon='magic',
            layout=widgets.Layout(width='100%')
        )
        
        self.btn_generate.on_click(self.on_generate_click)
        
        # Initial Render
        self.render()
        
        # Attempt to load model immediately
        self.load_model()

    def load_model(self):
        """Loads the GPT-2 Model with visual feedback"""
        if not HAS_LIB:
            with self.output:
                print("ERROR: Transformers/Torch libraries not found.")
                print("Please run: !pip install transformers torch ipywidgets")
            return

        self.loading_bar.layout.visibility = 'visible'
        self.loading_bar.value = 0.2
        
        with self.output:
            print("⏳ Loading GPT-2 model... (First time run may take a minute)")
        
        try:
            self.tokenizer = AutoTokenizer.from_pretrained("gpt2")
            self.loading_bar.value = 0.6
            self.model = AutoModelForCausalLM.from_pretrained("gpt2")
            self.loading_bar.value = 1.0
            
            self.is_loaded = True
            self.loading_bar.layout.visibility = 'hidden'
            
            with self.output:
                clear_output()
                print("✅ Model Loaded Successfully! Ready to generate.")
                
        except Exception as e:
            self.loading_bar.bar_style = 'danger'
            with self.output:
                print(f"❌ Error loading model: {str(e)}")

    def on_generate_click(self, b):
        """Handle button click event"""
        with self.output:
            if not HAS_LIB:
                print(f"❌ Library Error: {LIB_ERROR_MSG}")
                return
                
            if not self.is_loaded:
                print("⚠️ Model is still loading, please wait...")
                return
                
            clear_output(wait=True)
            print(f"📝 Prompt: '{self.input_text.value}'")
            print("⚙️ Generating...")
            
            try:
                # Encode
                inputs = self.tokenizer(self.input_text.value, return_tensors="pt")
                pad_token_id = self.tokenizer.eos_token_id
                
                # Generate
                outputs = self.model.generate(
                    inputs.input_ids,
                    max_length=self.length_slider.value,
                    num_return_sequences=1,
                    pad_token_id=pad_token_id,
                    do_sample=True,
                    top_k=50
                )
                
                # Decode
                text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                
                clear_output(wait=True)
                print("="*60)
                print("✨ GENERATED RESULT")
                print("="*60)
                print(text)
                print("="*60)
                
            except Exception as e:
                print(f"❌ Generation Error: {str(e)}")

    def render(self):
        """Display the UI"""
        display(widgets.HTML("<h2>🤖 GPT-2 Interactive Lab</h2>"))
        display(widgets.VBox([
            self.loading_bar,
            self.input_text,
            self.length_slider,
            self.btn_generate,
            self.output
        ]))

# Run App
app = GPT2App()

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\users\user\AppData\Roaming\Python\Python312\site-packages\torch\lib\c10.dll" or one of its dependencies.